# Fisheries Data Cleaning

In this notebook, i will share the code and method i use to load, check and clean the data set. the original data set i get from querying in [FAO data](https://www.fao.org/fishery/statistics-query/en/global_production/global_production_quantity). after that i get the data in csv and using it here. The main thing that i get is country, the amount of production, the species of fish, and continent as large scale comparison. 

## Import the data

In [4]:
import pandas as pd

path='path/to/the/file/Global_production_quantity_after_query.csv'

df=pd.read_csv(path)

### Check the head of dataframe

In [5]:
df.head()

,Country Name En,Continent Name En,ASFIS species Name En,Unit Name,2022,2022 Flag,2021,2021 Flag,2020,2020 Flag,...,1954,1954 Flag,1953,1953 Flag,1952,1952 Flag,1951,1951 Flag,1950,1950 Flag
0,Afghanistan,Asia,Cyprinids NEI,Tonnes - live weight,11100.0,E,11067.0,E,9404.0,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,Asia,Rainbow trout,Tonnes - live weight,50.0,E,40.0,E,40.0,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,Europe,Bighead carp,Tonnes - live weight,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,Europe,Common carp,Tonnes - live weight,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Albania,Europe,Crucian carp,Tonnes - live weight,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Since the data is on the wide shape, the first thing i do is melt it with df.melt(), so i can get fewer columns to work with

### Melt the data

In [7]:
id_vars = ['Country Name En', 'Continent Name En', 'ASFIS species Name En', 'Unit Name']
value_vars = [col for col in df.columns if col.isdigit()]

df_long = df.melt(
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='Year',
    value_name='Production'
)

#### Check the head of new df

In [8]:
df_long.head()

,Country Name En,Continent Name En,ASFIS species Name En,Unit Name,Year,Production
0,Afghanistan,Asia,Cyprinids NEI,Tonnes - live weight,2022,11100.0
1,Afghanistan,Asia,Rainbow trout,Tonnes - live weight,2022,50.0
2,Albania,Europe,Bighead carp,Tonnes - live weight,2022,NaN
3,Albania,Europe,Common carp,Tonnes - live weight,2022,NaN
4,Albania,Europe,Crucian carp,Tonnes - live weight,2022,NaN


### Clean the year and production data type

In [21]:
df_long['Year'] = df_long['Year'].astype(int)
df_long['Production'] = pd.to_numeric(df_long['Production'], errors='coerce')

### Check the empty value

In [9]:
df_long.isna().sum()

Country Name En               0
Continent Name En          1387
ASFIS species Name En         0
Unit Name                     0
Year                          0
Production               154668
dtype: int64

the production have a large of number that have nothing in it, so i decided to drop it down

#### Drop the empty row in production column 

In [10]:
df_long = df_long.dropna(subset=['Production'])

#### Check again the new df

In [11]:
df_long.isna().sum()

Country Name En            0
Continent Name En        317
ASFIS species Name En      0
Unit Name                  0
Year                       0
Production                 0
dtype: int64

I see that there are some empty value in Continent Name En, so i decided to check out what country is it or it is just an error

#### Check the continent name en that have nan value

In [12]:
df_long[df_long['Continent Name En'].isna()]

,Country Name En,Continent Name En,ASFIS species Name En,Unit Name,Year,Production
108867,Un. Sov. Soc. Rep.,NaN,Bighead carp,Tonnes - live weight,1991,5381.0
108868,Un. Sov. Soc. Rep.,NaN,Brown seaweeds,Tonnes - live weight,1991,9742.0
108869,Un. Sov. Soc. Rep.,NaN,Chum(=Keta=Dog) salmon,Tonnes - live weight,1991,0.0
108870,Un. Sov. Soc. Rep.,NaN,Common carp,Tonnes - live weight,1991,154427.0
108871,Un. Sov. Soc. Rep.,NaN,Flatfishes NEI,Tonnes - live weight,1991,71.0
...,...,...,...,...,...,...
248516,Un. Sov. Soc. Rep.,NaN,Common carp,Tonnes - live weight,1950,51500.0
248520,Un. Sov. Soc. Rep.,NaN,Mullets NEI,Tonnes - live weight,1950,0.0
248526,Un. Sov. Soc. Rep.,NaN,Silver carp,Tonnes - live weight,1950,0.0
248530,Un. Sov. Soc. Rep.,NaN,Whitefishes NEI,Tonnes - live weight,1950,2570.0


Since my main intention is to analyze the newer data and not thinking much about the history of the country, so i decided to exclude it from the main dataset and created a new dataset for it, if i needed it later

#### Save USSR rows to a separate DataFrame

In [13]:
df_ussr = df_long[df_long['Country Name En'] == 'Un. Sov. Soc. Rep.']

#### Exclude USSR from the main DataFrame

In [14]:
df_long_cleaned = df_long[df_long['Country Name En'] != 'Un. Sov. Soc. Rep.']

I ask chatgpt about what thing i should do to re-check if the data is already clean, so, here what it says.

✅ 1. Check Overall Structure


```
df.info()
```

Look for:
Year should be int64
Production should be float64 (or at least numeric)
No unexpected object types

In [22]:
df_long_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93653 entries, 0 to 248619
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country Name En        93653 non-null  object 
 1   Continent Name En      93653 non-null  object 
 2   ASFIS species Name En  93653 non-null  object 
 3   Unit Name              93653 non-null  object 
 4   Year                   93653 non-null  int64  
 5   Production             93653 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 5.0+ MB


✅ 2. Preview a Sample

```
df.sample(5)
```

Make sure:

Each row represents one country-species-unit-year

Production values are filled or marked as NaN (not weird strings)

In [23]:
df_long_cleaned.sample(5)

,Country Name En,Continent Name En,ASFIS species Name En,Unit Name,Year,Production
233383,Malaysia,Asia,Mozambique tilapia,Tonnes - live weight,1954,0.0
49015,Indonesia,Asia,Giant river prawn,Tonnes - live weight,2008,942.0
75898,Falkland Islands (Malvinas),Americas,Blue mussel,Tonnes - live weight,2000,0.0
111373,Republic of Korea,Asia,Marine fishes NEI,Tonnes - live weight,1990,929.0
100375,Kenya,Africa,Common carp,Tonnes - live weight,1993,289.0


✅ 3. Check for Missing Values

```
df.isna().sum()
```

This shows which columns still have missing data. A small number of NaNs might be okay, but a bunch could be a red flag.

In [24]:
df_long_cleaned.isna().sum()

Country Name En          0
Continent Name En        0
ASFIS species Name En    0
Unit Name                0
Year                     0
Production               0
dtype: int64

✅ 4. Check for Duplicates

```
df.duplicated().sum()
```

If this gives you anything above 0, you probably have redundant rows.

To inspect:
```
df[df.duplicated()]

In [25]:
df_long_cleaned.duplicated().sum()

np.int64(0)

✅ 5. Unique Values Sanity Check

```
df['Columns'].unique()
df['Columns'].nunique()
```
We're checking that if the value is make sense


In [27]:
df_long_cleaned['Year'].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012,
       2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001,
       2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990,
       1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979,
       1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969, 1968,
       1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958, 1957,
       1956, 1955, 1954, 1953, 1952, 1951, 1950])

In [31]:
df_long_cleaned['Country Name En'].nunique()

213

In [32]:
df_long_cleaned['Country Name En'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia (Plurinational State of)',
       'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'British Virgin Islands',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Channel Islands', 'Chile',
       'China', 'China, Hong Kong SAR', 'Colombia', 'Congo',
       'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba',
       'Cyprus', 'Czechia', "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
   

In [29]:
df_long_cleaned['ASFIS species Name En'].nunique()

731

In [33]:
df_long_cleaned['ASFIS species Name En'].unique()

array(['Cyprinids NEI', 'Rainbow trout', 'European seabass',
       'Gilthead seabream', 'Mediterranean mussel', 'Barbel', 'Bleak',
       'Common carp', 'Common sole', 'Crucian carp', 'European eel',
       'Freshwater bream', 'Freshwater fishes NEI',
       'Largemouth black bass', 'Mullets NEI', 'North African catfish',
       'Pacific cupped oyster', 'Penaeus shrimps NEI', 'Pike-perch',
       'Roach', 'Tench', 'Tilapias NEI', 'Nile tilapia',
       '[Eucheumatopsis isiformis]', '[Brycon orbignyanus]', 'Dorado',
       'Grass carp(=White amur)', '[Megaleporinus obtusidens]', 'Pacu',
       'Silver, bighead carps NEI', 'Sorubims NEI',
       'South American catfish', 'Streaked prochilod', 'Silver carp',
       'Sturgeons NEI', 'Trouts NEI', 'Abalones NEI', 'Atlantic salmon',
       'Australian mussel', 'Barramundi(=Giant seaperch)',
       'Flat and cupped oysters NEI', 'Marine fishes NEI',
       'Marine molluscs NEI', 'Murray cod', 'Red claw crayfish',
       'Sea trout', 'Silver 

✅ 6. Confirm Production is Numeric

```
df['Production'].dtype
df['Production'].describe()
```

If you see very small or very large numbers (e.g., 1e12), double-check if they are real or errors.

In [34]:
df_long_cleaned['Production'].dtype

dtype('float64')

In [35]:
df_long_cleaned['Production'].describe()

count    9.365300e+04
mean     2.625252e+04
std      2.409026e+05
min      0.000000e+00
25%      0.000000e+00
50%      4.213900e+01
75%      1.200000e+03
max      1.177848e+07
Name: Production, dtype: float64

With all of that alreadt checked out, i think the data is already clean and is ready to export as new csv for analyze later

## Export the csv

In [ ]:
df_long_cleaned.to_csv('the/path/of/the/file/df_cleaned.csv', index=False)

In [ ]:
df_ussr.to_csv('the/path/of/the/file/ussr_data.csv', index=False)

i think thats all for the cleaning, and i will continue for the next step


## 🤝 Credits

Created by tojangeng262, a fisheries graduate transitioning into data analytics.

This project is part of a professional data portfolio aimed at showcasing real-world data wrangling skills using Python, pandas, and GitHub best practices.
